In [87]:
import sqlite3
import matplotlib as plt
from tqdm import tqdm
import pandas as pd

## Database Design

For this assignment, we’ll focus on three pathways: **glycolysis, the citric acid cycle, and the pentose phosphate pathway**. We will consider genes in these pathways from **Drosophila, E. coli, and humans**.

 - *Gene Table*: name, description, organism, and nucleotide sequence. Additional fields might include chromosome, start and end position, strand, and translated sequence. For eukaryotes, the nucleotide sequence should be the spliced mRNA and the coordinates should span the entire locus.

 - *Pathway Table*: name and description...

 - *Enzyme Table:* name, function, and enzyme commission (EC) number. Multiple genes encode enzymes that perform the same function, so there ought to be fewer enzymes than genes.

## Declaration

In [90]:
conn = sqlite3.connect('lab4.db')
c = conn.cursor()

for tbl in ('genes', 'pathways', 'enzymes'):
    c.execute('DROP TABLE IF EXISTS %s;' % tbl);
c.execute('''CREATE TABLE genes (
    id TEXT,
    short TEXT,
    long TEXT,
    position TEXT,
    organism TEXT
);''')
c.execute('''CREATE TABLE pathways (
    keggMap TEXT,
    amiGO TEXT,
    name TEXT,
    description TEXT
);''')
c.execute('''CREATE TABLE enzymes (
    ec TEXT,
    name TEXT,
    description TEXT,
    cas TEXT
);''')

conn.commit()

## Accessing databases

Map of glycolysis from KEGG: https://www.genome.jp/kegg-bin/show_pathway?map00010

Databases
 - UCSC: https://genome.ucsc.edu/
 - Entrez: https://www.ncbi.nlm.nih.gov/search/
 - Reactome: https://reactome.org/
 - Pfam: https://pfam.xfam.org/
 - KEGG: https://www.genome.jp/kegg/

Pick 4 enzymes from glycolysis, TCA cycle, and pentose phosphate. Gather information and sequences from Drosophila, E. coli, and human for those enzymes.

Do you have any preference? If you’re clever, you can automate this entire process using Bio.Entrez, which can be used to pull entire annotations for each enzyme from each organism—and more. Try asking it to return data as GenBank and using `Bio.SeqIO` to read the returned GenBank file.

## Dataset building

You’ll be creating 4 enzymes * 3 pathways * 3 organisms = 36 rows in your gene table. In your pathways table, you should have a row per pathway (3 in total). In your enzymes table, you should have one row per enzyme (12 in total).

For this lab, I have decided to pull soley from KEGG using the utilities already in BioPython. Given an ID, it is possible to retrieve enzyme and gene features, but not pathways. So, I start with the manually retrieved information about the pathways and a sample of enzyme ID's associated with each.

In [74]:
# keggMap amiGO name description
pathways = [
    # https://www.genome.jp/dbget-bin/www_bget?map00020
    ('map00020', '0006099', 'Citrate cycle (TCA cycle)',
     'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.'),
    # https://www.genome.jp/dbget-bin/www_bget?map00030
    ('map00030', '0006098', 'Pentose phosphate pathway',
     'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.'),
    # https://www.genome.jp/dbget-bin/www_bget?map00010
    ('map00010', '0006096', 'Glycolysis / Gluconeogenesis',
     'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).'),
]
# you can try REST.kegg_get(KEGGmap), however I didn't have enough time to write a parser

#txid
organisms = [
    # https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Undef&name=Homo+sapiens&lvl=0&srchmode=1&keep=1&unlock
    ('hsa', 9606, 'Homo sapiens (human)'),
    # https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Undef&name=Escherichia+coli&lvl=0&srchmode=1&keep=1&unlock
    ('eco', 562, 'Escherichia coli K-12 MG1655'),
    # https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=7215
    ('dsa', 7215, 'Drosophila melanogaster (fruit fly)')
]

# https://www.genome.jp/dbget-bin/get_linkdb?-t+enzyme+path:map00020
# https://www.genome.jp/dbget-bin/get_linkdb?-t+enzyme+path:map00030
# https://www.genome.jp/dbget-bin/get_linkdb?-t+enzyme+path:map00010
ecs = [
    "2.7.1.11",
    "6.2.1.5",
    "1.8.1.4",
    "5.1.3.1",
    "4.1.2.4",
    "5.4.2.2",
    "4.2.1.3",
    "1.2.4.1",
    "1.1.1.44",
    "5.3.1.9",
    "4.2.1.11",
    "2.7.1.2",
]

In [75]:
from Bio import SeqIO
from Bio.KEGG import REST, Enzyme, Gene

def fetch_enzymes(ecs):
    '''Accesses a list of enzyme information from KEGG given EC numbers.'''
    records = []
    for i in tqdm(range(0, len(ecs), 10)):
        handles = REST.kegg_get(["ec:"+ec for ec in ecs[i:i+10]])
        records.extend(list(Enzyme.parse(handles)))
    return records

def fetch_genes(ids):
    '''Accesses gene information from KEGG given org:id format.'''
    records = []
    for i in tqdm(range(0, len(ids), 10)):
        handles = REST.kegg_get(ids[i:i+10])
        records.extend(list(Gene.parse(handles)))
    return list(records)

def entrez_lookup(query, limit=1):
    '''Looks up a query using Entrez.'''
    handle = Entrez.esearch(db = 'nucleotide',
                            term = 'query',
                            sort = 'relevance',
                            idtype = 'acc',
                            retmax = limit)
        
    for entry in Entrez.read(handle)['IdList']:  
        gb = Entrez.efetch(db='nucleotide', id=entry, rettype='gb', retmode = 'text')
        record = SeqIO.read(gb, 'gb')
        print(record.name, record.description, record.seq)

In [76]:
# https://biopython.org/docs/1.75/api/Bio.KEGG.Enzyme.html
r = fetch_enzymes(['5.4.2.2'])
r[0].__dict__.keys()

100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


dict_keys(['entry', 'name', 'classname', 'sysname', 'reaction', 'substrate', 'product', 'inhibitor', 'cofactor', 'effector', 'comment', 'pathway', 'genes', 'disease', 'structures', 'dblinks'])

In [60]:
# https://biopython.org/docs/1.75/api/Bio.KEGG.Gene.html
r = fetch_genes(['hsa:55276'])
r[0].__dict__.keys()

100%|██████████| 1/1 [00:00<00:00,  2.32it/s]


dict_keys(['entry', 'name', 'definition', 'orthology', 'organism', 'position', 'motif', 'dblinks'])

In [ ]:
def shorten_description(text):
    '''Keeps the first two sentences of `text`'''
    return '. '.join(text.split('. ')[:2]).rstrip('.')+'.'

enzymes = []
# ec id
enz_genes = []
# ec map
enz_path = []
for r in fetch_enzymes(ecs):
    enzymes.append((
        r.entry,
        r.name[0],
        shorten_description(r.comment[0]) if len(r.comment) != 0 else '',
        r.dblinks[-1][1][0],
    ))
    genes = dict(r.genes)
    for org in ('HSA', 'DME', 'ECO'):
        for gene in genes.get(org, []):
            enz_genes.append((r.entry, f'{org.lower()}:{gene}'))
    for p in r.pathway:
        if p[1][-5:] in {'00010', '00020', '00030'}:
            enz_path.append((r.entry, 'map'+p[1][-5:]))
enzymes, enz_genes, enz_path

In [ ]:
genes = []
for r in fetch_genes([id for ec, id in enz_genes]):
    genes.append((
        r.entry,
        r.name[0] if len(r.name) != 0 else None,
        r.definition,
        r.position,
        r.organism[0]
    ))
genes

In [91]:
c = conn.cursor()
c.executemany('INSERT INTO genes VALUES (?,?,?,?,?);', genes)
c.executemany('INSERT INTO pathways VALUES (?,?,?,?);', pathways)
c.executemany('INSERT INTO enzymes VALUES (?,?,?,?);', enzymes)
conn.commit()

In [97]:
pd.read_sql('SELECT * FROM pathways', conn)

,keggMap,amiGO,name,description
0,map00020,0006099,Citrate cycle (TCA cycle),"The citrate cycle (TCA cycle, Krebs cycle) is ..."
1,map00030,0006098,Pentose phosphate pathway,The pentose phosphate pathway is a process of ...
2,map00010,0006096,Glycolysis / Gluconeogenesis,Glycolysis is the process of converting glucos...


In [96]:
pd.read_sql('SELECT * FROM enzymes', conn)

,ec,name,description,cas
0,2.7.1.11,6-phosphofructokinase,D-Tagatose 6-phosphate and sedoheptulose 7-pho...,9001-80-3
1,6.2.1.5,succinate---CoA ligase (ADP-forming),,9080-33-5
2,1.8.1.4,dihydrolipoyl dehydrogenase,A flavoprotein (FAD). A component of the multi...,9001-18-7
3,5.1.3.1,ribulose-phosphate 3-epimerase,The enzyme also converts D-erythrose 4-phospha...,9024-20-8
4,4.1.2.4,deoxyribose-phosphate aldolase,,9026-97-5
5,5.4.2.2,"phosphoglucomutase (alpha-D-glucose-1,6-bispho...",Maximum activity is only obtained in the prese...,9001-81-4
6,4.2.1.3,aconitate hydratase,Besides interconverting citrate and cis-aconit...,9024-25-3
7,1.2.4.1,pyruvate dehydrogenase (acetyl-transferring),Contains thiamine diphosphate. It is a compone...,9014-20-4
8,1.1.1.44,phosphogluconate dehydrogenase (NADP+-dependen...,The enzyme participates in the oxidative branc...,9073-95-4
9,5.3.1.9,glucose-6-phosphate isomerase,Also catalyses the anomerization of D-glucose ...,9001-41-6


In [95]:
pd.read_sql('SELECT * FROM genes', conn)

,id,short,long,position,organism
0,5211,"PFKL, ATP-PFK, PFK-B, PFK-L","(RefSeq) phosphofructokinase, liver type",21q22.3,hsa
1,5213,"PFKM, ATP-PFK, GSD7, PFK-1, PFK-A, PFK1, PFKA,...","(RefSeq) phosphofructokinase, muscle",12q13.11,hsa
2,5214,"PFKP, ATP-PFK, PFK-C, PFK-P, PFKF","(RefSeq) phosphofructokinase, platelet",10p15.2,hsa
3,Dmel_CG4001,Pfk,"(RefSeq) phosphofructokinase, isoform B",2R,dme
4,b1723,pfkB,(RefSeq) 6-phosphofructokinase II,1806370..1807299,eco
...,...,...,...,...,...
56,387712,"ENO4, C10orf134",(RefSeq) enolase 4,10q25.3,hsa
57,Dmel_CG17654,Eno,"(RefSeq) enolase, isoform B",2L,dme
58,b2779,eno,(RefSeq) enolase,complement(2906643..2907941),eco
59,2645,"GCK, FGQTL3, GK, GLK, HHF3, HK4, HKIV, HXKP, L...",(RefSeq) glucokinase,7p13,hsa


## Relational modeling

For this part of the lab we would like to retrieve the enzyme information associated with each gene and the gene information associated with each pathway. Could this be accomplished another way than by adding additional fields to each table? Are the joins one-to-one, many-to-one, or one-to-many?

In general, joins in biology are many-to-many, as for example due to the complex, multi-step nature of pathways, any given enzyme could be involved in many pathways, while at the same time pathways require many enzymes as cofactors.

In [98]:
for tbl in ('enzyme_pathways', 'enzyme_genes'):
    c.execute('DROP TABLE IF EXISTS %s;' % tbl);

c.execute('''CREATE TABLE enzyme_pathways (
    ec TEXT,
    keggMap TEXT
);''')
c.executemany('INSERT INTO enzyme_pathways VALUES (?,?);', enz_path)

c.execute('''CREATE TABLE enzyme_genes (
    ec TEXT,
    id TEXT
);''')
c.executemany('INSERT INTO enzyme_genes VALUES (?,?);', enz_genes)

conn.commit()

In [100]:
pd.read_sql('SELECT * FROM enzyme_pathways', conn)

,ec,keggMap
0,2.7.1.11,map00010
1,2.7.1.11,map00030
2,6.2.1.5,map00020
3,1.8.1.4,map00010
4,1.8.1.4,map00020
5,5.1.3.1,map00030
6,4.1.2.4,map00030
7,5.4.2.2,map00010
8,5.4.2.2,map00030
9,4.2.1.3,map00020


In [101]:
pd.read_sql('SELECT * FROM enzyme_genes', conn)

,ec,id
0,2.7.1.11,hsa:5211
1,2.7.1.11,hsa:5213
2,2.7.1.11,hsa:5214
3,2.7.1.11,dme:Dmel_CG4001
4,2.7.1.11,eco:b1723
...,...,...
56,4.2.1.11,hsa:387712
57,4.2.1.11,dme:Dmel_CG17654
58,4.2.1.11,eco:b2779
59,2.7.1.2,hsa:2645
